In [1]:
from bs4 import BeautifulSoup
import requests
import json
import boto3
import scrape_site
from send_email import send_email

In [2]:
r = requests.get('https://humaneanimalrescue.org/adopt/?_type=dog')
main_page = BeautifulSoup(r.text, 'html.parser')
all_dogs = scrape_site.list_dogs(main_page)

with open('data/dogs.json', 'w') as f:
    json.dump(all_dogs, f, indent=4)

In [3]:
with open('data/dogs.json', 'r') as f:
    old_dogs = json.load(f)[1:]

In [4]:
old_dog_ids = [dog['id'] for dog in old_dogs]
new_dogs = []
for dog in all_dogs:
    if dog['id'] not in old_dog_ids:
        new_dogs.append(dog)

In [5]:
# get details for all new dogs
detailed_dogs = []
for dog in new_dogs:
    r = requests.get(dog['url'])
    details_page = BeautifulSoup(r.text, 'html.parser')
    detailed_dog = scrape_site.get_dog_details(details_page)
    detailed_dogs.append(detailed_dog)

In [6]:
detailed_dogs

[{'headshot': 'https://www.shelterluv.com/sites/default/files/animal_pics/17549/2022/04/22/08/20220422085901.png',
  'name': 'Agatha',
  'breed': 'Mixed Breed (Large)',
  'sex': 'Female',
  'age': '8 years 4 months',
  'weight': '62 lbs',
  'description': "It doesn't take a super sleuth to see how cute Agatha is. Like any true queen, she prefers all of the attention on her so she needs a home where she is the only pet. Agatha has a smile that you won't be able to resist. She has a few medical issues that we'll be happy to discuss with you. None of them stop her from living her best life. Agatha would prefer a calmer home to relax and be pampered in. Stop in to see her today!",
  'pictures': ['https://www.shelterluv.com/sites/default/files/animal_pics/17549/2022/03/05/11/20220305110010.png',
   'https://www.shelterluv.com/sites/default/files/animal_pics/17549/2022/03/22/10/20220322100851.png',
   'https://www.shelterluv.com/sites/default/files/animal_pics/17549/2022/03/30/09/20220330095

In [10]:
send_email('rob.mitchellzone@gmail.com', detailed_dogs)